## 0. 라이브러리

In [1]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import numpy as np

## 1. 데이터 불러오기

In [2]:
train_data_path ="~/aiffel/dktc/data2/train0.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data

,class,conversation
idx,,
1,일반 대화,야 여행 가기 너무 좋은 날씨다\n날씨 진짜 너무 좋아 **이는 일하기 싫을 날씨야...
2,갈취 대화,사정이 있었어요 . 살려주세요 . 뒤진 사람들 중에 사정 없는 사람 없어 맞아 . ...
3,갈취 대화,배고프다 그러게 야 만원만 빌려줘봐 오늘도 ? 저번에도 만원 빌려가고 안 돌려 줬잖...
4,갈취 대화,저기 실례지만 제가 휠체어를 타는 장애인이라서 아 혼자 휠체어 운전하시는게 어렵다고...
5,직장 내 괴롭힘 대화,김 사원 낙하산이라는 소문이 있던데 ? 요즘 시대에도 낙하산이 있어요 ? 요즘 더 ...
...,...,...
4826,일반 대화,티비는 잘 안 보니?\n티비 매일 보지\n너는?\n키키 요즘 핫한 스우파 안 보니?...
4827,갈취 대화,이거 나이키 신상이네 ? 응 아빠가 사주셨어 우와 멋지네 근데 너랑은 별로 안어울린...
4828,직장 내 괴롭힘 대화,일처리를 어떻게 한거야 ! 기간 내에 상품이 못오면 그사이 손실은 어떻게 할거냐고 ...


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [3]:
def preprocess_sentence(sentence):
    # synolp
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    #sentence = re.sub(r'[^\w\s]', '', sentence)
    # base preprocess
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 엔터 구분 (\n)
    sentence = sentence.replace("\n", "<EOL>")
    sentence = sentence.strip()
    return sentence

### 2.1-2 전처리 함수 적용

In [4]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))

100%|██████████| 4830/4830 [00:01<00:00, 3199.34it/s]


### 2.2 최대 길이 지정

In [5]:
MAX_LEN = 250

### 2.3 class(label) 인코딩

In [6]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화','일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

len(labels)

4830

In [7]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)

Class mapping: {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}


In [8]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, 
                                    max_length=max_seq_len, 
                                    pad_to_max_length=True,
                                   )
        
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len
        
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)
    
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    
    data_labels = np.asarray(data_labels, dtype=np.int32)
    
    return (input_ids, attention_masks, token_type_ids), data_labels

### 2.4 train-val

In [9]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42, stratify=labels)

In [10]:
# train data 증강

def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return "".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = (
        new_words[random_idx_2],
        new_words[random_idx_1],
    )
    return new_words


def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


print("before data augmentation: ", len(train_sentences))

train_splted = pd.DataFrame({ "sentence": train_sentences, "class": train_labels })

# random deletion
train_splted_rd = train_splted.copy()
train_splted_rd["sentence"] = train_splted_rd["sentence"].apply(random_deletion)

# random swap
train_splted_rs = train_splted.copy()

# with data augmentation
train_concated = pd.concat([train_splted , train_splted_rd , train_splted_rs])

print("after data augmentation: ", len(train_concated))

train_concated

before data augmentation:  3864
after data augmentation:  11592


,sentence,class
idx,,
346,김비서 오늘 스케줄 어떻게 되지 ? 네 오전에 바이어 미팅있고 점심후엔 임원회의 예...,3
1779,이거 빨리 끝내고 일해야겠다 ㅎ 넌 몇 시 출근 ? 난 새벽 시 반 . . . 울고...,2
3214,김 과장 내가 부탁한 자료는 어떻게 된거야 ? 아 네 부장님 아직 하고 있습니다 ....,3
4225,학생 이것 좀 도와줄 수 있어 ? 네 . ? 어떤 아니 우리가 밖에서 뭘 하는데 밖...,0
295,내가 지난번에 말한대로 좋게 말할 때 시키는대로 해 . 왜그래 . 나는 못하겠어 ....,4
...,...,...
2779,야 술 받으라고 ! 죄송합니다 . 한 잔 받겠습니다 . 너는 어째 회사 안이나 밖이...,3
2786,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...,1
2932,지민이 포기해 . 애초에 너희 어울리지 않아 . 그건 지민이랑 제가 판단하는겁니다 ...,4


In [11]:
train_sentences, train_labels = train_concated["sentence"], train_concated["class"]
print(len(train_sentences))

11592


## 3. 모델
### 3.1-1 토크나이저 정의

In [12]:
# BERT 토크나이저와 모델 준비
model_name = "klue/bert-base"

tokenizer = BertTokenizer.from_pretrained(model_name)
special_tokens_dict = {'additional_special_tokens': ['<EOL>']}
tokenizer.add_special_tokens(special_tokens_dict)

1

### 3.1-2 토크나이저 적용

In [13]:
# 데이터셋을 BERT 입력 형식으로 변환
X_train, y_train = convert_examples_to_features(
    train_sentences, train_labels, 
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)
X_valid, y_valid = convert_examples_to_features(
    val_sentences, val_labels, 
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)

# train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=MAX_LEN) # 뒤쪽에 패딩
# val_encodings = tokenizer(val_sentences, truncation=True, padding=True, max_length=MAX_LEN)

  0%|          | 0/11592 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 966/966 [00:00<00:00, 1037.13it/s]


### 3.2 모델 준비

In [14]:
class TFBertForMultiClassClassification(tf.keras.Model):
    def __init__(self, model_name, num_classes, dropout_rate=0.1):
        super(TFBertForMultiClassClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.classifier = tf.keras.layers.Dense(num_classes,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                kernel_regularizer=l2(0.01),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        cls_token = outputs[1]
        dropped = self.dropout(cls_token)
        prediction = self.classifier(dropped)
        return prediction

In [15]:
model = TFBertForMultiClassClassification(model_name, num_classes=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

### 3.3 파라미터

In [16]:
BATCH_SIZE = 16
lr = 5e-5
EPOCH = 10

### 3.4 TF 데이터셋 생성

In [17]:
# TensorFlow 데이터셋 생성
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_train),
#     y_train
# )).shuffle(100).batch(BATCH_SIZE)

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_valid),
#     y_valid
# )).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [19]:
import datetime 

early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=2,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

now = datetime.datetime.now()
time = now.strftime("%y-%m-%d %H:%M")
data_type = 0

checkpoint = ModelCheckpoint(
    filepath=f'./models/klue_with_aug_weights_{data_type}_m250_{time}.keras',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,    # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)

### 3.6-2 모델 훈련

In [20]:
model.fit(
    X_train, y_train, 
    validation_data=(X_valid, y_valid),
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/10
363/363 [==============================] - 613s 2s/step - loss: 0.3987 - accuracy: 0.8888 - val_loss: 0.3163 - val_accuracy: 0.9286

Epoch 00001: val_loss improved from inf to 0.31630, saving model to ./models/klue_with_aug_weights_0_m250_24-06-26 05:54.keras
Epoch 2/10
363/363 [==============================] - 596s 2s/step - loss: 0.1563 - accuracy: 0.9767 - val_loss: 0.3850 - val_accuracy: 0.9182

Epoch 00002: val_loss did not improve from 0.31630
Epoch 3/10
363/363 [==============================] - 596s 2s/step - loss: 0.1099 - accuracy: 0.9898 - val_loss: 0.4133 - val_accuracy: 0.9110

Epoch 00003: val_loss did not improve from 0.31630


### 3.7 모델 평가

In [21]:
# 모델 평가
evaluation = model.evaluate(X_valid, y_valid)
print("평가 결과:", evaluation)

31/31 [==============================] - 17s 533ms/step - loss: 0.3163 - accuracy: 0.9286
평가 결과: [0.3162969946861267, 0.9285714030265808]


In [22]:
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np 

def score(model, val):
    X, y = val
    # 실제 예측값 생성
    real_predictions = model.predict(X)

    # 예측값을 레이블로 변환
    real_predicted_labels = np.argmax(real_predictions, axis=1)

    # 정확도 계산
    real_accuracy = accuracy_score(y, real_predicted_labels)
    print(f"Real Accuracy: {real_accuracy:.4f}")

    # 분류 보고서 생성
    real_report = classification_report(y, real_predicted_labels, target_names=[f"Class {i}" for i in range(5)])
    print(real_report)

    # F1 스코어 계산
    real_f1 = f1_score(y, real_predicted_labels, average='weighted')
    print(f"\nWeighted F1 Score (based on real predictions): {real_f1:.4f}")

In [23]:
score(model, (X_valid, y_valid))

Real Accuracy: 0.9286
              precision    recall  f1-score   support

     Class 0       0.87      0.85      0.86       196
     Class 1       0.89      0.92      0.90       219
     Class 2       1.00      1.00      1.00       176
     Class 3       0.98      0.97      0.98       196
     Class 4       0.91      0.91      0.91       179

    accuracy                           0.93       966
   macro avg       0.93      0.93      0.93       966
weighted avg       0.93      0.93      0.93       966


Weighted F1 Score (based on real predictions): 0.9285


## 4. 모델 적용

In [24]:
import json

test_data_path = "/aiffel/aiffel/dktc/data/test.json"
test = pd.read_json(test_data_path).transpose()

In [25]:
import numpy as np

test_predict = []

for idx, value in test.iterrows():

    test_sentence = value["text"]
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="tf")

    test_predictions = model.predict(
        (test_encodings["input_ids"],
         test_encodings["attention_mask"],
         test_encodings["token_type_ids"])
    )
    test_class_probabilities = tf.nn.softmax(test_predictions, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predict.append(test_predicted_class[0])

In [26]:
# {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}
#   협박 대화 : 0,  갈취 대화 : 1,  직장 내 괴롭힘 대화 : 2,  기타 괴롭힘 대화 : 3,  일반 대화 : 4
def labelnum_to_text(x):
    if x == 0:
        return '01'
    if x == 1:
        return '03'
    if x == 2:
        return '04'
    if x == 3:
        return '02'
    if x == 4:
        return '00'

import datetime
    
submission = pd.read_csv("../data/new_submission.csv")
submission["class"] = [ labelnum_to_text(pred) for pred in test_predict ]

now = datetime.datetime.now()
filename = now.strftime("../sub/submission %y-%m-%d %H:%M.csv")

submission.to_csv(filename, index=False)
submit_file = pd.read_csv(filename)

print(submit_file.shape)
print(submit_file.head())

(500, 2)
  file_name  class
0     t_000      1
1     t_001      2
2     t_002      2
3     t_003      3
4     t_004      3


In [27]:
# submission 